In [1]:
import torch
import numpy as np
from ucimlrepo import fetch_ucirepo 
from sklearn.metrics import accuracy_score

from neural_blueprints.architectures import MLP
from neural_blueprints.config.architectures import MLPConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections.input_projections import TabularInputProjectionConfig
from neural_blueprints.config.components.composite.projections.output_projections import TabularOutputProjectionConfig
from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import MaskedTabularDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_ucirepo(id=2)
X = data.data.features
y = data.data.targets

data = X.copy()
data['income'] = y['income'].str.replace('.', '')     # Some target values have a trailing dot

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data = data[0]
data.head()

2025-12-21 19:14:36,257 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-21 19:14:36,258 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,8,0.044131,10,5,5,2,2,5,2,0.02174,0.0,0.397959,40,1
1,0.452055,7,0.048052,10,5,3,5,1,5,2,0.00000,0.0,0.122449,40,1
2,0.287671,5,0.137581,12,16,1,7,2,5,2,0.00000,0.0,0.397959,40,1
3,0.493151,5,0.150486,2,14,3,7,1,3,2,0.00000,0.0,0.397959,40,1
4,0.150685,5,0.220635,10,5,3,11,6,3,1,0.00000,0.0,0.397959,6,1


In [4]:
dataset = MaskedTabularDataset(
    data = data,
    discrete_features = discrete_features,
    continuous_features = continuous_features,
    mask_column='income'
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [12]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    normalization="batchnorm1d",
    activation='gelu',
    final_activation="relu",
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=16,
        normalization="batchnorm1d",
        activation="gelu"
    ),
    output_projection=TabularOutputProjectionConfig(
        cardinalities=[2],
        latent_dim=16,
        hidden_dims=[8],
        activation="gelu",
        normalization="batchnorm1d",
        final_activation="softmax"
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-21 19:19:05,146 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection
2025-12-21 19:19:05,147 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=240, hidden_dims=[128, 64, 32, 16], output_dim=16, normalization='batchnorm1d', activation='gelu', final_activation='relu', input_projection=TabularInputProjectionConfig(cardinalities=[1, 10, 1, 16, 16, 7, 16, 6, 5, 2, 1, 1, 1, 43, 2], hidden_dims=[64, 32], output_dim=16, dropout_p=None, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(cardinalities=[2], latent_dim=16, hidden_dims=[8], activation='gelu', normalization='batchnorm1d', dropout_p=None, final_activation='softmax'))

In [14]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="cross_entropy",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=20)

2025-12-21 19:19:13,574 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:07<02:17,  7.23s/epoch]

Epoch 1/20, Training Loss: 0.8805, Validation Loss: 0.7106


Training Epochs:  10%|█         | 2/20 [00:16<02:26,  8.15s/epoch]

...

Training Epochs:  60%|██████    | 12/20 [01:44<01:10,  8.82s/epoch]

Epoch 12/20, Training Loss: 0.6880, Validation Loss: 0.6846


Training Epochs:  65%|██████▌   | 13/20 [01:53<01:01,  8.84s/epoch]

Epoch 13/20, Training Loss: 0.6883, Validation Loss: 0.6841


2025-12-21 19:21:15,904 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 5 consecutive epochs. Early stopping at epoch 14.
Training Epochs:  65%|██████▌   | 13/20 [02:02<01:05,  9.41s/epoch]
2025-12-21 19:21:15,906 - neural_blueprints.utils.trainer - INFO - Training completed in 122.33 seconds.
2025-12-21 19:21:15,907 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.8345e-01


In [10]:
with torch.no_grad():
    X = val_dataset[:][0]
    y = val_dataset[:][1]
    y_pred = np.argmax(model(X)[0].numpy(), axis=1)
    print(f"Predicted: {y_pred[:20]},\nActual: {y[:20]}")
    print(y_pred.shape, y.shape)
    accuracy = accuracy_score(np.round(y_pred), y)
    print(f"Accuracy: {accuracy}")

Predicted: [2 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 2],
Actual: tensor([1., 1., 2., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
        1., 2.])
(9769,) torch.Size([9769])
Accuracy: 0.8727607738765483
